In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("InferSchema").getOrCreate()
sc = spark.sparkContext

In [8]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")

"""
inferschema vs predefind schema
inferSchema -
PredefineSchema - create a structType or case class define schema
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True) \
                .option("delimiter","|") \
                .csv(filepath + "IntUSPopulation.csv")
df.show()

+---------+----------------+----------+-------------+-----------+------------------+
|2019_rank|            City|State_Code|2019_estimate|2010_Census|            Change|
+---------+----------------+----------+-------------+-----------+------------------+
|        1|     New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|     Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|         Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4|      Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|         Phoenix|        AZ|      1680992|    1445632|            0.1628|
|        6|     San Antonio|        TX|      1547253|    1327407|            0.1656|
|        7|       San Diego|        CA|      1423851|    1307402|            0.0891|
|        8|          Dallas|        TX|      1343573|    1197816|            0.1217|
|        9|        San Jose|        CA|      1021795|     945942|

In [9]:
#Method1
#inferSchema(read thru entire file) is used to get datatype for all columns. Read.csv is neither transformation not action
#spark job triggered to create a DAG visualisation to read entire file
df1 = spark.read.option("header",True) \
                .option("delimiter","|") \
                .csv(filepath + "IntUSPopulation.csv",inferSchema=True)
df.show()

+---------+----------------+----------+-------------+-----------+------------------+
|2019_rank|            City|State_Code|2019_estimate|2010_Census|            Change|
+---------+----------------+----------+-------------+-----------+------------------+
|        1|     New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|     Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|         Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4|      Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|         Phoenix|        AZ|      1680992|    1445632|            0.1628|
|        6|     San Antonio|        TX|      1547253|    1327407|            0.1656|
|        7|       San Diego|        CA|      1423851|    1307402|            0.0891|
|        8|          Dallas|        TX|      1343573|    1197816|            0.1217|
|        9|        San Jose|        CA|      1021795|     945942|

In [12]:
schema = StructType([
    StructField("2019_rank",IntegerType(),True),
    StructField("City",StringType(),True),
    StructField("State_code",StringType(),True),
    StructField("2019_estimate",IntegerType(),True),
    StructField("2010_Census",IntegerType(),True),
    StructField("Change",StringType(),True)
])

In [14]:
#Method2
#No spark job created, fastest run
df1 = spark.read.option("header",True) \
                .option("delimiter","|") \
                .schema(schema) \
                .csv(filepath + "IntUSPopulation.csv")
df1.show()

+---------+----------------+----------+-------------+-----------+------------------+
|2019_rank|            City|State_code|2019_estimate|2010_Census|            Change|
+---------+----------------+----------+-------------+-----------+------------------+
|        1|     New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|     Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|         Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4|      Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|         Phoenix|        AZ|      1680992|    1445632|            0.1628|
|        6|     San Antonio|        TX|      1547253|    1327407|            0.1656|
|        7|       San Diego|        CA|      1423851|    1307402|            0.0891|
|        8|          Dallas|        TX|      1343573|    1197816|            0.1217|
|        9|        San Jose|        CA|      1021795|     945942|